In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/icr-identify-age-related-conditions/sample_submission.csv
/kaggle/input/icr-identify-age-related-conditions/greeks.csv
/kaggle/input/icr-identify-age-related-conditions/train.csv
/kaggle/input/icr-identify-age-related-conditions/test.csv


In [2]:
!pip install optuna

In [3]:
df_train = pd.read_csv("/kaggle/input/icr-identify-age-related-conditions/train.csv")
df_test = pd.read_csv("/kaggle/input/icr-identify-age-related-conditions/test.csv")
dt_greeks = pd.read_csv("/kaggle/input/icr-identify-age-related-conditions/greeks.csv")

In [4]:
#Left join with Greeks file
df_train_greeks = pd.merge(df_train, dt_greeks, on='Id', how='left')

# Cleanning / Tranforming

In [5]:
#Change column EJ to binary 1/0
first_category = df_train.EJ.unique()[0]
df_train.EJ = df_train.EJ.eq(first_category).astype('int')
df_train_greeks.EJ = df_train_greeks.EJ.eq(first_category).astype('int')
df_test.EJ = df_test.EJ.eq(first_category).astype('int')

#Change column Alpha to binary 1/0
first_category_alpha = 'A'
df_train_greeks.Alpha = df_train_greeks.Alpha.eq(first_category_alpha).astype('int')

In [6]:
from sklearn.preprocessing import RobustScaler, MinMaxScaler
from sklearn.impute import KNNImputer, SimpleImputer

# Separate features and target variable
X = df_train.drop(['Class', 'Id'], axis=1)
y = df_train['Class']

# Impute missing values with median for numerical attributes
num_imputer = SimpleImputer(strategy='median')
# Scale numerical attributes
scaler = RobustScaler()

X_train_scaled = pd.DataFrame(scaler.fit_transform(num_imputer.fit_transform(X)), columns=X.columns)

In [7]:
X_train_scaled.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 617 entries, 0 to 616
Data columns (total 56 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AB      617 non-null    float64
 1   AF      617 non-null    float64
 2   AH      617 non-null    float64
 3   AM      617 non-null    float64
 4   AR      617 non-null    float64
 5   AX      617 non-null    float64
 6   AY      617 non-null    float64
 7   AZ      617 non-null    float64
 8   BC      617 non-null    float64
 9   BD      617 non-null    float64
 10  BN      617 non-null    float64
 11  BP      617 non-null    float64
 12  BQ      617 non-null    float64
 13  BR      617 non-null    float64
 14  BZ      617 non-null    float64
 15  CB      617 non-null    float64
 16  CC      617 non-null    float64
 17  CD      617 non-null    float64
 18  CF      617 non-null    float64
 19  CH      617 non-null    float64
 20  CL      617 non-null    float64
 21  CR      617 non-null    float64
 22  CS

# Keras

In [8]:
def balanced_log_loss(y_true, y_pred):
    # y_true: correct labels 0, 1
    # y_pred: predicted probabilities of class=1
    # calculate the number of observations for each class
    N_0 = np.sum(1 - y_true)
    N_1 = np.sum(y_true)
    # calculate the weights for each class to balance classes
    w_0 = 1 / N_0
    w_1 = 1 / N_1
    # calculate the predicted probabilities for each class
    p_1 = np.clip(y_pred, 1e-15, 1 - 1e-15)
    p_0 = 1 - p_1
    # calculate the summed log loss for each class
    log_loss_0 = -np.sum((1 - y_true) * np.log(p_0))
    log_loss_1 = -np.sum(y_true * np.log(p_1))
    # calculate the weighted summed logarithmic loss
    # (factgor of 2 included to give same result as LL with balanced input)
    balanced_log_loss = 2*(w_0 * log_loss_0 + w_1 * log_loss_1) / (w_0 + w_1)
    # return the average log loss
    return balanced_log_loss/(N_0+N_1)

In [9]:
##### import xgboost as xgb
import optuna
import lightgbm as lgb
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow.keras.backend as K
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, f1_score
from sklearn.metrics import make_scorer, log_loss
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_score, StratifiedKFold, RepeatedStratifiedKFold
from sklearn.preprocessing import RobustScaler, MinMaxScaler
from sklearn.impute import KNNImputer, SimpleImputer
import warnings

# Disable LightGBM warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)



# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_train_scaled, y, test_size=0.2, random_state=42)
    
    
def objective(trial):
     # Define the search space for hyperparameters
    units = trial.suggest_int('units', 32, 256, log=True)
    activation = trial.suggest_categorical('activation', ['relu', 'sigmoid', 'tanh'])
    optimizer = trial.suggest_categorical('optimizer', ['adam', 'sgd'])

    # Define the Keras model
    model = keras.Sequential([
        layers.Dense(units=units, activation=activation, input_shape=(56,)),
        layers.Dense(units=units, activation=activation),
        layers.Dense(units=1, activation='sigmoid')
    ])

    # Compile the model
    model.compile(optimizer=optimizer,
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    
    # Train the model
    model.fit(X_train, y_train, batch_size=32, epochs=10, validation_split=0.2)

    # Evaluate the model
    # Evaluate the model
    y_pred = model.predict(X_test)
    # Calculate log loss
    logloss = log_loss(y_test, y_pred)

    # Return the log loss
    return logloss

# Create a study object for optimization
study = optuna.create_study(direction='minimize')

# Start the optimization process
study.optimize(objective, n_trials=100)

# Print the best hyperparameters found
best_params = study.best_params
print('Best Hyperparameters: ', best_params)


/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

Epoch 1/10
13/13 [==============================] - 1s 34ms/step - loss: 0.4664 - accuracy: 0.8223 - val_loss: 0.4184 - val_accuracy: 0.8485
Epoch 2/10
13/13 [==============================] - 0s 8ms/step - loss: 0.4307 - accuracy: 0.8223 - val_loss: 0.3923 - val_accuracy: 0.8485
Epoch 3/10
13/13 [==============================] - 0s 7ms/step - loss: 0.4016 - accuracy: 0.8198 - val_loss: 0.3886 - val_accuracy: 0.8485
Epoch 4/10
13/13 [==============================] - 0s 7ms/step - loss: 0.3746 - accuracy: 0.8223 - val_loss: 0.3680 - val_accuracy: 0.8586
Epoch 5/10
13/13 [==============================] - 0s 7ms/step - loss: 0.3451 - accuracy: 0.8452 - val_loss: 0.3533 - val_accuracy: 0.8485
Epoch 6/10
13/13 [==============================] - 0s 8ms/step - loss: 0.3143 - accuracy: 0.8528 - val_loss: 0.3406 - val_accuracy: 0.8485
Epoch 7/10
13/13 [==============================] - 0s 7ms/step - loss: 0.2888 - accuracy: 0.8782 - val_loss: 0.3237 - val_accuracy: 0.8586
Epoch 8/10
13/13 [=

[I 2023-07-13 00:42:27,732] Trial 0 finished with value: 0.302256089316216 and parameters: {'units': 115, 'activation': 'sigmoid', 'optimizer': 'adam'}. Best is trial 0 with value: 0.302256089316216.


Epoch 1/10
13/13 [==============================] - 1s 22ms/step - loss: 0.9845 - accuracy: 0.3249 - val_loss: 0.5859 - val_accuracy: 0.8485
Epoch 2/10
13/13 [==============================] - 0s 6ms/step - loss: 0.5346 - accuracy: 0.8223 - val_loss: 0.4650 - val_accuracy: 0.8485
Epoch 3/10
13/13 [==============================] - 0s 6ms/step - loss: 0.4847 - accuracy: 0.8223 - val_loss: 0.4360 - val_accuracy: 0.8485
Epoch 4/10
13/13 [==============================] - 0s 6ms/step - loss: 0.4746 - accuracy: 0.8223 - val_loss: 0.4323 - val_accuracy: 0.8485
Epoch 5/10
13/13 [==============================] - 0s 6ms/step - loss: 0.4737 - accuracy: 0.8223 - val_loss: 0.4266 - val_accuracy: 0.8485
Epoch 6/10
13/13 [==============================] - 0s 6ms/step - loss: 0.4710 - accuracy: 0.8223 - val_loss: 0.4257 - val_accuracy: 0.8485
Epoch 7/10
13/13 [==============================] - 0s 6ms/step - loss: 0.4708 - accuracy: 0.8223 - val_loss: 0.4243 - val_accuracy: 0.8485
Epoch 8/10
13/13 [=

[I 2023-07-13 00:42:29,699] Trial 1 finished with value: 0.4776350931691368 and parameters: {'units': 134, 'activation': 'sigmoid', 'optimizer': 'sgd'}. Best is trial 0 with value: 0.302256089316216.


...

Epoch 1/10
13/13 [==============================] - 1s 23ms/step - loss: 0.5064 - accuracy: 0.7766 - val_loss: 0.4182 - val_accuracy: 0.8485
Epoch 2/10
13/13 [==============================] - 0s 7ms/step - loss: 0.4405 - accuracy: 0.8223 - val_loss: 0.4035 - val_accuracy: 0.8485
Epoch 3/10
13/13 [==============================] - 0s 7ms/step - loss: 0.4109 - accuracy: 0.8223 - val_loss: 0.3864 - val_accuracy: 0.8485
Epoch 4/10
13/13 [==============================] - 0s 7ms/step - loss: 0.3766 - accuracy: 0.8223 - val_loss: 0.3665 - val_accuracy: 0.8586
Epoch 5/10
13/13 [==============================] - 0s 6ms/step - loss: 0.3378 - accuracy: 0.8452 - val_loss: 0.3532 - val_accuracy: 0.8485
Epoch 6/10
13/13 [==============================] - 0s 7ms/step - loss: 0.3107 - accuracy: 0.8655 - val_loss: 0.3317 - val_accuracy: 0.8485
Epoch 7/10
13/13 [==============================] - 0s 6ms/step - loss: 0.2832 - accuracy: 0.8959 - val_loss: 0.3161 - val_accuracy: 0.8586
Epoch 8/10
13/13 [=

[I 2023-07-13 00:46:25,919] Trial 98 finished with value: 0.3135231550351983 and parameters: {'units': 163, 'activation': 'sigmoid', 'optimizer': 'adam'}. Best is trial 93 with value: 0.283198146535718.


Epoch 1/10
13/13 [==============================] - 1s 24ms/step - loss: 0.5053 - accuracy: 0.7766 - val_loss: 0.3935 - val_accuracy: 0.8182
Epoch 2/10
13/13 [==============================] - 0s 9ms/step - loss: 0.3046 - accuracy: 0.8909 - val_loss: 0.3860 - val_accuracy: 0.8182
Epoch 3/10
13/13 [==============================] - 0s 7ms/step - loss: 0.2427 - accuracy: 0.8959 - val_loss: 0.3315 - val_accuracy: 0.8687
Epoch 4/10
13/13 [==============================] - 0s 8ms/step - loss: 0.2064 - accuracy: 0.9264 - val_loss: 0.3732 - val_accuracy: 0.8485
Epoch 5/10
13/13 [==============================] - 0s 7ms/step - loss: 0.1996 - accuracy: 0.9416 - val_loss: 0.3741 - val_accuracy: 0.8889
Epoch 6/10
13/13 [==============================] - 0s 7ms/step - loss: 0.1685 - accuracy: 0.9289 - val_loss: 0.3502 - val_accuracy: 0.8889
Epoch 7/10
13/13 [==============================] - 0s 8ms/step - loss: 0.1251 - accuracy: 0.9594 - val_loss: 0.3613 - val_accuracy: 0.8788
Epoch 8/10
13/13 [=

[I 2023-07-13 00:46:28,486] Trial 99 finished with value: 0.39263671989484034 and parameters: {'units': 243, 'activation': 'tanh', 'optimizer': 'adam'}. Best is trial 93 with value: 0.283198146535718.


Best Hyperparameters:  {'units': 190, 'activation': 'sigmoid', 'optimizer': 'adam'}


# Cross - Validation

In [10]:
final_model = keras.Sequential([
    layers.Dense(units=best_params['units'], activation=best_params['activation'], input_shape=(56,)),
    layers.Dense(units=best_params['units'], activation=best_params['activation']),
    layers.Dense(units=1, activation='sigmoid')
])
final_model.compile(optimizer=best_params['optimizer'],
                    loss='binary_crossentropy',
                    metrics=['accuracy'])
final_model.fit(X_train, y_train, batch_size=32, epochs=10, validation_split=0.2)


Epoch 1/10
13/13 [==============================] - 1s 24ms/step - loss: 0.5633 - accuracy: 0.7310 - val_loss: 0.4354 - val_accuracy: 0.8485
Epoch 2/10
13/13 [==============================] - 0s 7ms/step - loss: 0.4664 - accuracy: 0.8223 - val_loss: 0.3959 - val_accuracy: 0.8485
Epoch 3/10
13/13 [==============================] - 0s 7ms/step - loss: 0.4201 - accuracy: 0.8223 - val_loss: 0.3837 - val_accuracy: 0.8485
Epoch 4/10
13/13 [==============================] - 0s 6ms/step - loss: 0.3830 - accuracy: 0.8223 - val_loss: 0.3643 - val_accuracy: 0.8485
Epoch 5/10
13/13 [==============================] - 0s 6ms/step - loss: 0.3505 - accuracy: 0.8376 - val_loss: 0.3496 - val_accuracy: 0.8586
Epoch 6/10
13/13 [==============================] - 0s 7ms/step - loss: 0.3209 - accuracy: 0.8452 - val_loss: 0.3315 - val_accuracy: 0.8485
Epoch 7/10
13/13 [==============================] - 0s 7ms/step - loss: 0.2922 - accuracy: 0.8756 - val_loss: 0.3204 - val_accuracy: 0.8485
Epoch 8/10
13/13 [=

In [11]:
#Predictions
features = [n for n in df_test.columns if n != 'Class' and n != 'Id']
test_imputed = num_imputer.transform(df_test[features])
test_scaled = scaler.transform(test_imputed)
final_test = final_model.predict(test_scaled)


submission = pd.DataFrame(df_test["Id"], columns=["Id"])
submission["class_0"] = 1-final_test 
submission["class_1"] = final_test
submission.to_csv('/kaggle/working/submission.csv', index=False)

1/1 [==============================] - 0s 69ms/step
